# Graphene SVM Testing

In [322]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random

from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA

# Read in the data from the numpy array files provided
pp = np.load('./perfect_patches.npy')
dp = np.load('./defect_patches.npy')

# Label the data as 1 for within boundary and -1 for outlier points as output by OneClassSVM
pp_labels = np.ones(len(pp))
dp_labels = np.ones(len(dp)) * -1

patches = np.concatenate([pp, dp])
gt = np.concatenate([pp_labels, dp_labels])

xy = list(zip(patches, gt))     # put labels with data in (data, label) tuple
random.shuffle(xy)              # shuffle the tuples
patches, gt = list(zip(*xy))    # unzip the tuples to get patches and ground truths as vectors
patches = np.array(patches)

# Reshape data for compatibility with sklearn as a 2d array (as opposed to 3d)
#nsamples_tr, nx_tr, ny_tr = pp.shape
#d2_train = pp.reshape((nsamples_tr, nx_tr*ny_tr))
#nsamples_test, nx_test, ny_test = dp.shape
#d2_test = dp.reshape((nsamples_test, nx_test*ny_test))

Define Model for OneClassSVM, train and test with the reshaped data for perfect patches and defect patches:

In [323]:
model = OneClassSVM(gamma='scale',
                    nu=0.005, kernel='rbf')
model1 = OneClassSVM(gamma='scale',
                    nu=0.005, kernel='poly')
model2 = OneClassSVM(gamma='scale',
                    nu=0.005, kernel='sigmoid')
model3 = OneClassSVM(gamma='scale',
                    nu=0.005, kernel='linear')

n_samples, nx, ny = patches.shape
d2 = patches.reshape((n_samples, nx*ny))

model.fit(d2)
label_predict = model.predict(d2)

model1.fit(d2)
label_predict1 = model1.predict(d2)

model2.fit(d2)
label_predict2 = model2.predict(d2)

model3.fit(d2)
label_predict3 = model3.predict(d2)

error_percent = (sum(gt - label_predict) / len(gt)) * 100
print(f'Training Error rbf: {error_percent:.2f} %')


#model.fit(d2_train)
#label_train = model.predict(d2_train)
#label_test = model.predict(d2_test)

#n_error_train = (label_train[label_train == -1].size / label_train.size) * 100
#n_error_test = (label_test[label_test == 1].size / label_test.size) * 100

#print(f'Training Error: {n_error_train} %')
#print(f'Testing Error: {n_error_test} %')

Training Error rbf: 1.38 %


Training Error is the percentage of the training data which is assigned a label which does not match the ground truth.

Use the full_stack data for testing the trained model.

In [324]:
graphene = np.load('./full-stack.npy')
full_stack_labels = []

for i in range(len(graphene)):
    smaller_patches = 0
    sample = np.squeeze(graphene[i])
    sample_patches = np.split(sample, 4, axis=0)

    for j in sample_patches:
        smaller_patches = np.split(j, 4, axis=1)
        smaller_patches = np.array(smaller_patches)

    smaller_samples, nx, ny = np.shape(smaller_patches)
    smaller_2d = smaller_patches.reshape((smaller_samples, nx*ny))
    smaller_2d.resize(4, 2304)
    smaller_predicted_labels = model.predict(smaller_2d)            # Breaks here since N features are incorrect
    print(smaller_predicted_labels)



ValueError: cannot resize this array: it does not own its data